In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.table import Table
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
from datetime import timedelta
from statistics import mode
from scipy.signal import find_peaks

from scipy.integrate import odeint

# Libraries for Correlations
import scipy.stats as stats
from scipy.stats import pearsonr, sem, variation, kruskal,f_oneway
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform

from sklearn.preprocessing import LabelEncoder

from itertools import combinations, product

%matplotlib inline

from scipy.optimize import minimize

from scipy.integrate import odeint

import random  # Add this import statement at the beginning of your script

import ast
from scipy.integrate import odeint
from scipy.interpolate import interp1d

from multiprocessing import Pool, cpu_count
from functools import partial
from itertools import product

############################################

Processing the Glucose Values to Isolate the PPGR

############################################

In [ ]:
def read_data(filename):
    # unfiltered = pd.read_csv(os.path.join(filename))
    # Use the detected 'MacRoman' encoding
    unfiltered = pd.read_csv(os.path.join(filename), encoding='MacRoman')
    unfiltered['glucose_level_ts'] = pd.to_datetime(unfiltered['glucose_level_ts'], dayfirst=True, errors='coerce')
    unfiltered['meal_ts'] = pd.to_datetime(unfiltered['meal_ts'], dayfirst=True, errors='coerce')
    unfiltered['bolus_ts'] = pd.to_datetime(unfiltered['bolus_ts'], dayfirst=True, errors='coerce')

    return unfiltered


In [ ]:
def interleave_arrays_increasing(meal, bolus):
    result = []
    i, j = 0, 0
    while i < len(meal) and j < len(bolus):
        if meal[i] < bolus[j]:
            result.append(("meal", meal[i]))
            i += 1
        else:
            result.append(("bolus", bolus[j]))
            j += 1
    result.extend([("meal", meal[x]) for x in range(i, len(meal))])
    result.extend([("bolus", bolus[x]) for x in range(j, len(bolus))])
    return result


In [ ]:
def glucoseForMealsTs(glucose_ts_array, meal_ts):
    closest_values = []
    for element in meal_ts:
        closest_index = np.abs(glucose_ts_array - element).argmin()
        closest_values.append(glucose_ts_array[closest_index])
    return np.array(closest_values)


In [ ]:
def glucoseForEventsTs(glucose_ts_array, events_ts):
    closest_values = []
    for event_type, event_ts in events_ts:
        closest_index = np.abs(glucose_ts_array - event_ts).argmin()
        closest_value = glucose_ts_array[closest_index]

        if abs(closest_value - event_ts) <= pd.Timedelta(hours=4):
            closest_values.append((event_type, event_ts, closest_value))

    return closest_values


In [ ]:
def bolusMealSeparation(meal_ts, bolus_start_ts, bolus_dose_filtered):
    viableBolusTimes = []
    for i in range(min(len(bolus_start_ts), len(bolus_dose_filtered))):
        isWithinRange = False
        for j in range(len(meal_ts)):
            if meal_ts[j] - pd.Timedelta(minutes=4) <= bolus_start_ts[i] <= meal_ts[j] + pd.Timedelta(hours=4):
                isWithinRange = True
                break
        if not isWithinRange:
            viableBolusTimes.append((bolus_start_ts[i], bolus_dose_filtered[i]))
    return viableBolusTimes


In [ ]:
def groupBolus2(bolus_array):
    time_ranges = [
        ("6am-10am", datetime.time(6, 0), datetime.time(10, 0)),
        ("10am-2pm", datetime.time(10, 0), datetime.time(14, 0)),
        ("2pm-6pm", datetime.time(14, 0), datetime.time(18, 0)),
        ("6pm-10pm", datetime.time(18, 0), datetime.time(22, 0)),
    ]

    result = []
    df = pd.DataFrame(bolus_array, columns=['Timestamp', 'Value'])
    grouped = df.groupby(df['Timestamp'].dt.date)

    for date, group_data in grouped:
        daily_result = {'Date': date, 'TimeRanges': []}
        for label, start_time, end_time in time_ranges:
            time_mask = (group_data['Timestamp'].dt.time >= start_time) & (group_data['Timestamp'].dt.time < end_time)
            max_value = group_data.loc[time_mask, 'Value'].max()
            max_timestamps = group_data.loc[(time_mask) & (group_data['Value'] == max_value), 'Timestamp'].tolist()
            daily_result['TimeRanges'].append({
                'TimeRange': label,
                'MaxValue': max_value,
                'Timestamps': max_timestamps
            })
        result.append(daily_result)
    return result


In [ ]:
def findTimestampsNotCoveredByMeals(result, meal_events):
    time_ranges = {
        "6am-10am": (datetime.time(6, 0), datetime.time(10, 0)),
        "10am-2pm": (datetime.time(10, 0), datetime.time(14, 0)),
        "2pm-6pm": (datetime.time(14, 0), datetime.time(18, 0)),
        "6pm-10pm": (datetime.time(18, 0), datetime.time(22, 0))
    }

    meal_events_set = set(meal_events)
    timestamps_not_covered = []

    for day_result in result:
        for time_range_result in day_result['TimeRanges']:
            time_range_label = time_range_result['TimeRange']
            timestamps = time_range_result['Timestamps']
            time_range_start, time_range_end = time_ranges[time_range_label]

            meal_events_within_range = False
            for meal in meal_events_set:
                meal_time = meal.time()
                time_range_start_datetime = datetime.datetime.combine(day_result['Date'], time_range_start)
                time_range_end_datetime = datetime.datetime.combine(day_result['Date'], time_range_end)

                if time_range_start_datetime <= meal <= time_range_end_datetime:
                    meal_events_within_range = True
                    break

            if not meal_events_within_range:
                timestamps_not_covered.extend(timestamps)
    return timestamps_not_covered


In [ ]:
def filter_glucose_levels(glucose_ts_array, glucose_level_array, event_ts):
    start_time = event_ts
    end_time = event_ts + pd.Timedelta(hours=4)
    filtered_glucose_levels = []
    previous_timestamp = None
    for ts, level in zip(glucose_ts_array, glucose_level_array):
        if start_time <= ts <= end_time:
            if previous_timestamp is not None and (ts - previous_timestamp) > pd.Timedelta(minutes=30):
                break
            filtered_glucose_levels.append(level)
            previous_timestamp = ts
    return filtered_glucose_levels


###################################

Read and format DataFrame

###################################

In [ ]:

# Read data from the original file
participant_file = 'UoM2404.csv'  # Update this with the actual file name

insulin_sensitivity_factor = 2.9

unfiltered = read_data(participant_file)
unfiltered.sort_values('glucose_level_ts', inplace=True)
unfiltered.reset_index(drop=True, inplace=True)

glucose_level_ts = pd.to_datetime(unfiltered['glucose_level_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
glucose_level = unfiltered['glucose_level'].copy().to_numpy()  # Already converted to mmol/L
bolus_ts = pd.to_datetime(unfiltered['bolus_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
meal_ts = pd.to_datetime(unfiltered['meal_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')

bolus_dose = unfiltered['bolus_dose'].copy().to_numpy()
carbs_g = unfiltered['carbs_g']
meal_tags = unfiltered['meal_tag']
meal_types = unfiltered['meal_Type']


In [ ]:

nan_mask = np.isnan(bolus_dose)
bolus_dose_filtered = bolus_dose[~nan_mask]

nat_mask = np.isnat(meal_ts)
meal_ts_filtered = meal_ts[~nat_mask]

nat_mask = np.isnat(bolus_ts)
bolus_start_ts_filtered = bolus_ts[~nat_mask]

closest_glucose_array_meals = glucoseForMealsTs(glucose_level_ts, meal_ts_filtered)

bolusAndValueArray = bolusMealSeparation(meal_ts_filtered, bolus_start_ts_filtered, bolus_dose_filtered)
max_bolus_time_range = groupBolus2(bolusAndValueArray)
bolus_replacement_array = findTimestampsNotCoveredByMeals(max_bolus_time_range, meal_ts_filtered)

interleaved_meal_bolus_array = interleave_arrays_increasing(meal_ts_filtered, bolus_replacement_array)
closest_glucose_meal_bolus_array = glucoseForEventsTs(glucose_level_ts, interleaved_meal_bolus_array)



In [ ]:
# Extract 'carbs_g' and 'meal_tags' from the original data
carbs_g = unfiltered['carbs_g']

data_points = []

for event_type, event_ts, glucose_ts in closest_glucose_meal_bolus_array:
    closest_index = np.abs(glucose_level_ts - glucose_ts).argmin()
    glucose_levels = glucose_level[closest_index:closest_index + 48]

    carbs_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'carbs_g'].values[0] if event_type == "meal" else None
    meal_tag_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'meal_tag'].values[0] if event_type == "meal" else None
    meal_type_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'meal_Type'].values[0] if event_type == "meal" else None
    bolus_dose_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'bolus_dose'].values[0] if event_type == "meal" else None
    bolus_time = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'bolus_ts'].values[0] if event_type == "meal" else None

    data_point = {
        "EventTimestamp": event_ts,
        "GlucoseLevels": glucose_levels,
        "EventType": event_type,
        "EventTag": carbs_value,
        "MealTag": meal_tag_value,
        "MealType": meal_type_value,
        "BolusTime" : bolus_time,
        "BolusDose" : bolus_dose_value
    }
    data_points.append(data_point)

# Assume data_points is already defined as your input DataFrame
GlucoseEvents_exploded_clean = pd.DataFrame(data_points)


In [ ]:

GlucoseEvents_exploded_clean['EventTimestamp'] = pd.to_datetime(GlucoseEvents_exploded_clean['EventTimestamp'])
GlucoseEvents_exploded_clean['day_of_the_week'] = GlucoseEvents_exploded_clean['EventTimestamp'].dt.dayofweek
GlucoseEvents_exploded_clean['hour'] = GlucoseEvents_exploded_clean['EventTimestamp'].dt.hour

# Ensure Meal_Type is prioritized for MealCategory
GlucoseEvents_exploded_clean['MealCategory'] = GlucoseEvents_exploded_clean['MealType']

# Assign time-based categories where MealType is not available
time_based_categories = pd.cut(
    GlucoseEvents_exploded_clean['hour'],
    bins=[0, 10, 16, 22],
    labels=['Breakfast', 'Lunch', 'Dinner'],
    right=False
)

GlucoseEvents_exploded_clean['MealCategory'].fillna(time_based_categories, inplace=True)

<ipython-input-14-d109c604871d>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  GlucoseEvents_exploded_clean['MealCategory'].fillna(time_based_categories, inplace=True)


######################################

Remove administered insulin

####################################

In [ ]:

# Define insulin action model parameters (Adjust if needed)
tau2 = 50       # Decay constant for plasma insulin
p2 = 0.02       # Insulin effect decay rate
SI = insulin_sensitivity_factor       # Insulin sensitivity (Adjust based on individual data)
dt = 5          # Time step in minutes (Assuming glucose readings every 5 min)


# Ensure 'NetGlucose' column exists and can store lists
GlucoseEvents_exploded_clean['NetGlucose'] = np.nan
GlucoseEvents_exploded_clean = GlucoseEvents_exploded_clean.astype({'NetGlucose': 'object'})  # Allow list storage

# Function to simulate insulin dynamics
def insulin_dynamics(y, t, bolus_doses, dose_times):
    IP, IEFF = y
    bolus_doses = np.array(bolus_doses)  # Convert to NumPy array
    dose_times = np.array(dose_times)    # Convert to NumPy array

    # Compute subcutaneous insulin concentration
    ISC = np.sum(bolus_doses * np.exp(-(t - dose_times) / tau2))

    # Differential equations
    dIP_dt = (-IP / tau2) + (ISC / tau2)
    dIEFF_dt = (-p2 * IEFF) + (p2 * SI * IP)

    return [dIP_dt, dIEFF_dt]

# Process each bolus event
for index, row in GlucoseEvents_exploded_clean.iterrows():
    if pd.notna(row['BolusDose']):
        # Extract timestamps and glucose values
        glucose_levels = np.array(row['GlucoseLevels'], dtype=np.float64)

        # Handle NaN values in glucose readings
        if np.any(np.isnan(glucose_levels)):
            continue  # Skip this event if glucose readings are all NaN

        bolus_dose = row['BolusDose']

        # Define simulation time steps
        num_steps = len(glucose_levels)
        time_steps = np.arange(0, num_steps * dt, dt)
        dose_times = np.array([0])  # Assume bolus given at time zero

        # Initial conditions
        y0 = [0, 0]  # Initial IP and IEFF

        # Solve the differential equations using numerical integration
        sol = odeint(insulin_dynamics, y0, time_steps, args=([bolus_dose], dose_times))

        # Extract insulin effect values
        IEFF_values = sol[:, 1]

        # **Fix length mismatch**
        if len(IEFF_values) > num_steps:
            IEFF_values = IEFF_values[:num_steps]  # Truncate excess values
        elif len(IEFF_values) < num_steps:
            IEFF_values = np.pad(IEFF_values, (0, num_steps - len(IEFF_values)), 'edge')  # Pad with last value

        # Compute NetGlucose
        GlucoseEvents_exploded_clean.at[index, 'NetGlucose'] = glucose_levels + IEFF_values  # Assign as NumPy array

# Display the modified DataFrame
print(GlucoseEvents_exploded_clean[['EventTimestamp', 'GlucoseLevels', 'NetGlucose']])


         EventTimestamp                                      GlucoseLevels  \
0   2024-03-10 08:41:00  [8.4, 5.7, 4.7, 4.4, 5.2, 6.3, 6.2, 6.3, 6.4, ...   
1   2024-03-10 12:19:00  [11.0, 11.9, 13.4, 14.6, 14.7, 14.2, 12.8, 11....   
2   2024-03-10 17:00:00  [10.5, 11.6, 12.1, 12.3, 12.4, 12.3, 11.6, 9.8...   
3   2024-03-11 07:00:00  [11.5, 10.7, 11.4, 11.6, 10.8, 10.3, 9.3, 8.9,...   
4   2024-03-11 09:30:00  [6.1, 6.3, 6.9, 7.3, 7.2, 6.7, 5.9, 5.3, 5.0, ...   
..                  ...                                                ...   
336 2024-06-03 20:45:00  [4.4, 5.7, 6.6, 7.5, 7.7, 5.8, 5.9, 7.4, 9.6, ...   
337 2024-06-04 08:00:00  [8.9, 7.5, 6.6, 5.7, 6.2, 6.9, 7.6, 8.4, 9.3, ...   
338 2024-06-04 11:30:00  [5.4, 4.9, 4.3, 3.8, 3.6, 3.7, 4.5, 6.1, 7.7, ...   
339 2024-06-04 18:30:00  [10.1, 9.8, 9.4, 9.3, 10.1, 10.6, 10.4, 10.5, ...   
340 2024-06-05 08:30:00  [10.9, 10.4, 9.6, 9.2, 9.1, 9.1, 10.2, 11.6, 1...   

                                            NetGlucose  
0     

##################################

Normalize PPGRs

#################################

In [ ]:

# Define the function to find the peak
def find_peak(arr):
    return max(arr)

# Define the function to adjust values around the peak
def adjust_values(arr):
    # Ensure that all elements are numeric
    arr = pd.to_numeric(arr, errors='coerce')  # Convert non-numeric values to NaN
    arr = arr[~np.isnan(arr)]  # Remove NaN values

    if len(arr) == 0:  # In case the array becomes empty after removing NaNs
        return []

    peak_index = np.argmax(arr)  # Find the index of the peak value
    start_index = max(0, peak_index - 5)
    end_index = min(len(arr), peak_index + 11)  # 10 values after the peak

    adjusted_values = arr[start_index:end_index]
    peak_value = np.max(adjusted_values)
    normalized_values = np.round(adjusted_values - peak_value, 2)
    return normalized_values.tolist()  # Return as list for compatibility

# Function to calculate the coefficient of variation
def calculate_cv(arr):
    # Convert to numeric values, coercing errors to NaN
    arr = pd.to_numeric(arr, errors='coerce')
    arr = arr[~np.isnan(arr)]  # Remove NaN values

    if len(arr) == 0:  # Check if the array is empty after removing NaNs
        return np.nan

    series = np.array(arr)
    if not np.isclose(series.mean(), 0):
        return (series.std() / series.mean()) * 100
    else:
        return np.nan

# Calculate the CV of concatenated arrays
def calculate_combined_cv(arr1, arr2):
    combined = np.concatenate((arr1, arr2))
    if combined.size > 0:
        return calculate_cv(combined)
    else:
        return np.nan

# Apply the functions to create new columns
GlucoseEvents_exploded_clean['PeakGlucose'] = GlucoseEvents_exploded_clean['GlucoseLevels'].apply(find_peak)
GlucoseEvents_exploded_clean['AdjustedGlucose'] = GlucoseEvents_exploded_clean['GlucoseLevels'].apply(adjust_values)
GlucoseEvents_exploded_clean['NormalizedGlucose'] = GlucoseEvents_exploded_clean['AdjustedGlucose']
GlucoseEvents_exploded_clean['CV'] = GlucoseEvents_exploded_clean['GlucoseLevels'].apply(calculate_cv)

# Sort DataFrame by 'EventTimestamp' if it's not already sorted
GlucoseEvents_exploded_clean.sort_values(by='EventTimestamp', inplace=True)


###################################

Group based off pairwise CV matching

Iterate through the 'NormalizedGlucose' taking one Event and matching to another one, if the CV of the 2 is <36% they stay matched in a group, this iterates through making sure each added event meets the CV target. When the target is not met the event gets excluded. then the excluded events are treated the same until all events are assigned a group.

###################################

In [ ]:

# # Function to calculate the CV between two glucose levels
# def calculate_cv_between_sets(set1, set2):
#     set1 = pd.to_numeric(set1, errors='coerce')
#     set2 = pd.to_numeric(set2, errors='coerce')

#     # Check if any value in the sets is NaN
#     if np.any(np.isnan(set1)) or np.any(np.isnan(set2)):
#         return np.nan

#     combined_set = np.concatenate([set1, set2])
    # return calculate_cv(combined_set)


def calculate_cv_between_sets(set1, set2):
    # Ensure both sets are numpy arrays
    set1 = np.array(set1, dtype=np.float64) if isinstance(set1, (list, np.ndarray)) else np.array([set1], dtype=np.float64)
    set2 = np.array(set2, dtype=np.float64) if isinstance(set2, (list, np.ndarray)) else np.array([set2], dtype=np.float64)

    # Remove NaN values
    set1 = set1[~np.isnan(set1)]
    set2 = set2[~np.isnan(set2)]

    # If either set is empty after NaN removal, return NaN
    if set1.size == 0 or set2.size == 0:
        return np.nan

    combined_set = np.concatenate([set1, set2])

    return calculate_cv(combined_set)

# # Initialize DataFrame to hold grouped data
# grouped_data = []
# excluded_count = 0

# # Process each MealCategory separately
# for meal_category, meal_group in GlucoseEvents_exploded_clean.groupby('MealCategory'):
#     meal_group = meal_group.reset_index(drop=True)
#     group_numbers = np.zeros(len(meal_group), dtype=int)
#     current_group = 1

#     for i in range(len(meal_group)):
#         if group_numbers[i] == 0:  # Process unassigned events
#             group_numbers[i] = current_group
#             for j in range(i + 1, len(meal_group)):
#                 if group_numbers[j] == 0:
#                     # Extract glucose levels (ensure they are numeric)
#                     glucose_level_i = pd.to_numeric(meal_group.at[i, 'NetGlucose'], errors='coerce')
#                     glucose_level_j = pd.to_numeric(meal_group.at[j, 'NetGlucose'], errors='coerce')

#                     # Ensure the glucose levels are numeric values (not lists or arrays)
#                     if isinstance(glucose_level_i, (list, np.ndarray)):
#                         glucose_level_i = np.array(glucose_level_i)  # Flatten to numpy array
#                     if isinstance(glucose_level_j, (list, np.ndarray)):
#                         glucose_level_j = np.array(glucose_level_j)  # Flatten to numpy array

#                     # Calculate the CV between these glucose levels
#                     within_group_cv = calculate_cv_between_sets(glucose_level_i, glucose_level_j)

#                     if within_group_cv < 36:
#                         temp_group = meal_group[group_numbers == current_group]
#                         temp_group = pd.concat([temp_group, meal_group.iloc[[j]]])
#                         overall_cv = calculate_cv(np.concatenate(temp_group['NetGlucose'].values))
#                         if overall_cv < 36:
#                             group_numbers[j] = current_group
#             current_group += 1

#     # After assigning group numbers, check the size of each cluster
#     meal_group['GroupNumber'] = group_numbers  # Update 'GroupNumber' column here

#     group_size = meal_group.groupby('GroupNumber').size()  # Now 'GroupNumber' should exist

#     # Assign outliers for groups with 1 PPGR event
#     for group_num in group_size.index:
#         if group_size[group_num] == 1:  # If the group size is 1, assign to outlier group
#             meal_group.loc[meal_group['GroupNumber'] == group_num, 'GroupNumber'] = -1  # Use -1 for outliers

#     unassigned_count = np.sum(group_numbers == 0)
#     excluded_count += unassigned_count

#     grouped_data.append(meal_group)

# # Concatenate all grouped data
# GlucoseEvents_exploded_clean = pd.concat(grouped_data)

# # Group by MealCategory and GroupNumber to get the count of events in each group
# group_counts = GlucoseEvents_exploded_clean.groupby(['MealCategory', 'GroupNumber']).size().reset_index(name='EventCount')

# # Print the updated DataFrame with outliers (GroupNumber = -1)
# print(GlucoseEvents_exploded_clean[['EventTimestamp', 'GlucoseLevels', 'NetGlucose', 'PeakGlucose', 'AdjustedGlucose', 'NormalizedGlucose', 'CV', 'GroupNumber']].head())

# Function to safely concatenate NetGlucose values
def safe_concatenate_netglucose(temp_group):
    # Ensure that all entries in 'NetGlucose' are arrays (flatten lists if necessary)
    net_glucose_values = []

    # Iterating over each meal category
    for glucose_value in temp_group['NetGlucose']:
        # If glucose_value is a list or array, flatten it to a 1D array
        if isinstance(glucose_value, (list, np.ndarray)):
            glucose_value = np.array(glucose_value).flatten()
        # If it's a scalar, make it a single-element array
        elif np.issubdtype(type(glucose_value), np.number):
            glucose_value = np.array([glucose_value])

        # Append to the list if it's not empty
        if glucose_value.size > 0:
            net_glucose_values.append(glucose_value)

    # Concatenate only the non-empty arrays
    if len(net_glucose_values) > 0:
        return np.concatenate(net_glucose_values)
    else:
        return np.array([])  # Return empty array if no valid values

# Initialize DataFrame to hold grouped data
grouped_data = []
excluded_count = 0

# Process each MealCategory separately
for meal_category, meal_group in GlucoseEvents_exploded_clean.groupby('MealCategory'):
    meal_group = meal_group.reset_index(drop=True)
    group_numbers = np.zeros(len(meal_group), dtype=int)
    current_group = 1

    for i in range(len(meal_group)):
        if group_numbers[i] == 0:  # Process unassigned events
            group_numbers[i] = current_group
            for j in range(i + 1, len(meal_group)):
                if group_numbers[j] == 0:
                    # Extract glucose levels (ensure they are numeric)
                    glucose_level_i = pd.to_numeric(meal_group.at[i, 'NetGlucose'], errors='coerce')
                    glucose_level_j = pd.to_numeric(meal_group.at[j, 'NetGlucose'], errors='coerce')

                    # Ensure the glucose levels are numeric values (not lists or arrays)
                    if isinstance(glucose_level_i, (list, np.ndarray)):
                        glucose_level_i = np.array(glucose_level_i)  # Flatten to numpy array
                    if isinstance(glucose_level_j, (list, np.ndarray)):
                        glucose_level_j = np.array(glucose_level_j)  # Flatten to numpy array

                    # Calculate the CV between these glucose levels
                    within_group_cv = calculate_cv_between_sets(glucose_level_i, glucose_level_j)

                    if within_group_cv < 36:
                        temp_group = meal_group[group_numbers == current_group]
                        temp_group = pd.concat([temp_group, meal_group.iloc[[j]]])
                        combined_net_glucose = safe_concatenate_netglucose(temp_group)

                        # Calculate the overall CV if combined_net_glucose is not empty
                        if combined_net_glucose.size > 0:
                            overall_cv = calculate_cv(combined_net_glucose)
                            if overall_cv < 36:
                                group_numbers[j] = current_group
            current_group += 1

    # After assigning group numbers, check the size of each cluster
    meal_group['GroupNumber'] = group_numbers  # Update 'GroupNumber' column here

    group_size = meal_group.groupby('GroupNumber').size()  # Now 'GroupNumber' should exist

    # Assign outliers for groups with 1 PPGR event
    for group_num in group_size.index:
        if group_size[group_num] == 1:  # If the group size is 1, assign to outlier group
            meal_group.loc[meal_group['GroupNumber'] == group_num, 'GroupNumber'] = -1  # Use -1 for outliers

    unassigned_count = np.sum(group_numbers == 0)
    excluded_count += unassigned_count

    grouped_data.append(meal_group)

# Concatenate all grouped data
GlucoseEvents_exploded_clean = pd.concat(grouped_data)

# Group by MealCategory and GroupNumber to get the count of events in each group
group_counts = GlucoseEvents_exploded_clean.groupby(['MealCategory', 'GroupNumber']).size().reset_index(name='EventCount')

# Print the updated DataFrame with outliers (GroupNumber = -1)
print(GlucoseEvents_exploded_clean[['EventTimestamp', 'GlucoseLevels', 'NetGlucose', 'PeakGlucose', 'AdjustedGlucose', 'NormalizedGlucose', 'CV', 'GroupNumber']].head())



       EventTimestamp                                      GlucoseLevels  \
0 2024-03-10 08:41:00  [8.4, 5.7, 4.7, 4.4, 5.2, 6.3, 6.2, 6.3, 6.4, ...   
1 2024-03-11 07:00:00  [11.5, 10.7, 11.4, 11.6, 10.8, 10.3, 9.3, 8.9,...   
2 2024-03-12 08:00:00  [7.7, 8.1, 7.3, 6.6, 6.3, 6.9, 8.5, 10.7, 12.1...   
3 2024-03-13 07:30:00  [5.2, 5.0, 6.3, 6.1, 5.3, 5.1, 4.6, 4.4, 4.7, ...   
4 2024-03-14 07:30:00  [6.1, 6.1, 6.6, 6.9, 7.1, 6.0, 4.4, 4.1, 5.1, ...   

                                          NetGlucose  PeakGlucose  \
0                                                NaN         14.7   
1  [11.5, 10.752480559344542, 11.589945530068807,...         15.2   
2  [7.7, 8.139360433816604, 7.442459164955409, 6....         12.1   
3  [5.2, 5.052480559344543, 6.489945530068806, 6....         10.6   
4  [6.1, 6.126240287270859, 6.6949727804864345, 7...         16.4   

                                     AdjustedGlucose  \
0  [-5.4, -3.7, -2.8, -1.3, -0.1, 0.0, -0.5, -1.9...   
1  [-6.8, -6.3, 

##############################

Cluster Stats

############################

In [ ]:

# Functions to calculate standard deviation and standard error
def calculate_standard_error(arr):
    arr = np.array(arr)
    return np.std(arr) / np.sqrt(len(arr)) if len(arr) > 1 else np.nan

def calculate_standard_deviation(arr):
    arr = np.array(arr)
    return np.std(arr)

# Extended function to calculate cohesion metrics for each group
def calculate_cohesion_metrics(group_df):
    # Convert 'GlucoseLevels' to numeric values, forcing non-numeric to NaN
    glucose_levels = pd.to_numeric(group_df['NetGlucose'].explode(), errors='coerce').dropna().astype(float)

    # Collect all MealTags within the group
    meal_tags = group_df['MealTag'].unique()  # Get unique MealTags for the group

    # Proceed only if glucose_levels is not empty
    if len(glucose_levels) == 0:
        cohesion_metrics = {
            'MeanPeak': np.nan,
            'StandardDeviation': np.nan,
            'CoefficientOfVariation': np.nan,
            'StandardError': np.nan,
            'EventCount': 0,
            'IsOutlier': True,  # Consider this as outlier because of missing data
            'MealTags': meal_tags  # Include the MealTags for the group
        }
    else:
        cohesion_metrics = {
            'MeanPeak': glucose_levels.mean(),
            'StandardDeviation': calculate_standard_deviation(glucose_levels),
            'CoefficientOfVariation': calculate_cv(glucose_levels),
            'StandardError': calculate_standard_error(glucose_levels),
            'EventCount': group_df['NetGlucose'].count(),
            'IsOutlier': group_df['NetGlucose'].count() <= 1,  # Flag as outlier if EventCount is 1 or lower
            'MealTags': meal_tags  # Include the MealTags for the group
        }

    return cohesion_metrics

# Group by MealCategory and GroupNumber
grouped = GlucoseEvents_exploded_clean.groupby(['MealCategory', 'GroupNumber'])
cohesion_metrics_list = []

# Iterate over each group and calculate cohesion metrics
for (meal_category, group_number), group_df in grouped:
    cohesion_metrics = calculate_cohesion_metrics(group_df)
    cohesion_metrics['MealCategory'] = meal_category
    cohesion_metrics['GroupNumber'] = group_number
    cohesion_metrics_list.append(cohesion_metrics)

# Convert the list of dictionaries to a DataFrame
cohesion_metrics_df = pd.DataFrame(cohesion_metrics_list)

# Merge 'MealTags' per group into the final DataFrame
mean_meal_tag_per_group = GlucoseEvents_exploded_clean.groupby(['MealCategory', 'GroupNumber'])['MealTag'].unique().reset_index()
mean_meal_tag_per_group.rename(columns={'MealTag': 'MealTags'}, inplace=True)

# Merge cohesion metrics with MealTags
cohesion_metrics_with_meals = pd.merge(cohesion_metrics_df, mean_meal_tag_per_group, on=['MealCategory', 'GroupNumber'], how='left')

# Now the cohesion_metrics_with_meals DataFrame contains all MealTags for each cluster

# Replace 'MeanCarbs' with 'MealTag' (no mean calculation needed for categorical data)
mean_meal_tag_per_group = GlucoseEvents_exploded_clean.groupby(['MealCategory', 'GroupNumber'])['MealTag'].first().reset_index()
mean_meal_tag_per_group.rename(columns={'MealTag': 'MeanMealTag'}, inplace=True)



In [ ]:

# Convert 'MealTag' into numeric labels
label_encoder = LabelEncoder()
GlucoseEvents_exploded_clean['MealTag_encoded'] = label_encoder.fit_transform(GlucoseEvents_exploded_clean['MealTag'])

# Modify the ANOVA part to use 'MealTag_encoded'
anova_results = []

def perform_anova_for_category(category):
    category_data = GlucoseEvents_exploded_clean[GlucoseEvents_exploded_clean['MealCategory'] == category]
    anova_data = []
    group_sizes = []  # Track the number of observations per group

    for group_number, group_df in category_data.groupby('GroupNumber'):
        meal_tag_values = group_df['MealTag_encoded'].dropna().values  # Use the encoded 'MealTag'
        if len(meal_tag_values) > 0:
            anova_data.append(meal_tag_values)
            group_sizes.append(len(meal_tag_values))  # Track the group size for each group

    if len(anova_data) > 1:
        anova_result = f_oneway(*anova_data)

        # Degrees of freedom calculations
        k = len(anova_data)  # Number of groups
        N = sum(group_sizes)  # Total number of observations
        df_between = k - 1
        df_within = N - k

        return anova_result, df_between, df_within
    else:
        return None, None, None

# Repeat the ANOVA analysis
meal_categories = GlucoseEvents_exploded_clean['MealCategory'].unique()

for category in meal_categories:
    result, df_between, df_within = perform_anova_for_category(category)
    if result is not None:
        anova_results.append({
            'MealCategory': category,
            'ANOVA_F_statistic': result.statistic,
            'ANOVA_p_value': result.pvalue,
            'df_between': df_between,
            'df_within': df_within
        })
    else:
        anova_results.append({
            'MealCategory': category,
            'ANOVA_F_statistic': np.nan,
            'ANOVA_p_value': np.nan,
            'df_between': np.nan,
            'df_within': np.nan
        })

# Convert results to DataFrame
anova_results_df = pd.DataFrame(anova_results)

# Merge 'MeanMealTag' per group with cohesion_metrics_df
cohesion_metrics_with_meals = pd.merge(cohesion_metrics_df, mean_meal_tag_per_group, on=['MealCategory', 'GroupNumber'], how='left')

# Merge ANOVA results with cohesion_metrics_df
cohesion_metrics_with_anova = pd.merge(cohesion_metrics_with_meals, anova_results_df, on='MealCategory', how='left')

# Compare standard error with CGM standard error (0.8 mmol/L)
cohesion_metrics_with_anova['SE_Comparison'] = cohesion_metrics_with_anova['StandardError'] > 0.8

# Calculate cluster error rate
cluster_error_rate = cohesion_metrics_with_anova['SE_Comparison'].mean()

# Add the cluster error rate to the final DataFrame
cohesion_metrics_with_anova['ClusterErrorRate'] = cluster_error_rate

# Display the final DataFrame
print(cohesion_metrics_with_anova)
print(f"Cluster Error Rate: {cluster_error_rate * 100:.2f}%")



    MeanPeak  StandardDeviation  CoefficientOfVariation  StandardError  \
0        NaN                NaN                     NaN            NaN   
1   9.388016           2.632589               28.042012       0.040278   
2   9.453231           4.359694               46.118560       0.629268   
3  11.400714           3.468216               30.421041       0.063069   
4   8.006146           2.109434               26.347681       0.073845   
5        NaN                NaN                     NaN            NaN   
6   9.990348           2.868973               28.717443       0.046298   
7  10.071335           4.603552               45.709456       0.297158   
8  10.281209           3.159246               30.728355       0.059875   
9  13.410629           4.465877               33.301029       0.372156   

   EventCount  IsOutlier                                           MealTags  \
0           0       True                                             [None]   
1          89      False  [

############################

Baseline Stats

############################

In [ ]:


GlucoseEvents_exploded_clean.head()



,EventTimestamp,GlucoseLevels,EventType,EventTag,MealTag,MealType,BolusTime,BolusDose,day_of_the_week,hour,MealCategory,NetGlucose,PeakGlucose,AdjustedGlucose,NormalizedGlucose,CV,GroupNumber,MealTag_encoded
0,2024-03-10 08:41:00,"[8.4, 5.7, 4.7, 4.4, 5.2, 6.3, 6.2, 6.3, 6.4, ...",bolus,NaN,None,None,NaT,NaN,6,8,Breakfast,NaN,14.7,"[-5.4, -3.7, -2.8, -1.3, -0.1, 0.0, -0.5, -1.9...","[-5.4, -3.7, -2.8, -1.3, -0.1, 0.0, -0.5, -1.9...",30.731714,-1,187
1,2024-03-11 07:00:00,"[11.5, 10.7, 11.4, 11.6, 10.8, 10.3, 9.3, 8.9,...",meal,34.0,Porridge,Breakfast,2024-03-10 08:41:00,4.0,0,7,Breakfast,"[11.5, 10.752480559344542, 11.589945530068807,...",15.2,"[-6.8, -6.3, -4.8, -3.1, -1.8, 0.0, -1.8, -1.8...","[-6.8, -6.3, -4.8, -3.1, -1.8, 0.0, -1.8, -1.8...",27.346319,2,127
2,2024-03-12 08:00:00,"[7.7, 8.1, 7.3, 6.6, 6.3, 6.9, 8.5, 10.7, 12.1...",meal,34.0,Porridge,Breakfast,2024-03-10 17:28:00,3.0,1,8,Breakfast,"[7.7, 8.139360433816604, 7.442459164955409, 6....",12.1,"[-5.5, -5.8, -5.2, -3.6, -1.4, 0.0, -0.2, -0.8...","[-5.5, -5.8, -5.2, -3.6, -1.4, 0.0, -0.2, -0.8...",21.508744,2,127
3,2024-03-13 07:30:00,"[5.2, 5.0, 6.3, 6.1, 5.3, 5.1, 4.6, 4.4, 4.7, ...",meal,34.0,Porridge,Breakfast,2024-03-12 09:02:00,4.0,2,7,Breakfast,"[5.2, 5.052480559344543, 6.489945530068806, 6....",10.6,"[-2.8, -2.7, -2.0, -0.9, -0.2, 0.0, -0.2, -0.3...","[-2.8, -2.7, -2.0, -0.9, -0.2, 0.0, -0.2, -0.3...",28.717764,2,127
4,2024-03-14 07:30:00,"[6.1, 6.1, 6.6, 6.9, 7.1, 6.0, 4.4, 4.1, 5.1, ...",meal,34.0,Porridge,Breakfast,2024-03-12 18:39:00,2.0,3,7,Breakfast,"[6.1, 6.126240287270859, 6.6949727804864345, 7...",16.4,"[-7.0, -5.6, -3.1, -1.3, -0.2, 0.0, -0.6, -0.8...","[-7.0, -5.6, -3.1, -1.3, -0.2, 0.0, -0.6, -0.8...",39.412411,2,127


In [ ]:
# Constants for glucose ranges
TIGHT_LOWER = 3.9  # Lower limit for tight range (mmol/L)
TIGHT_UPPER = 7.8  # Upper limit for tight range (mmol/L)
LOWER_TARGET = 3.9  # Lower limit of target range (mmol/L)
UPPER_TARGET = 10.0  # Upper limit of target range (mmol/L)

# Function to calculate time in various ranges
def calculate_time_in_ranges(glucose_levels):
    # Ensure the input is a pandas Series to handle non-numeric entries
    if isinstance(glucose_levels, np.ndarray):
        glucose_levels_series = pd.Series(glucose_levels)
    else:
        glucose_levels_series = pd.Series(glucose_levels)

    # Convert glucose levels to numeric values, forcing invalid strings to NaN
    glucose_levels_series = pd.to_numeric(glucose_levels_series, errors='coerce')

    # Drop any NaN values that result from invalid conversions
    glucose_levels_clean = glucose_levels_series.dropna()

    # Convert to numpy array for further calculations
    glucose_levels_array = glucose_levels_clean.to_numpy()

    # Masks for different ranges
    in_target_range = (glucose_levels_array >= LOWER_TARGET) & (glucose_levels_array <= UPPER_TARGET)
    in_tight_range = (glucose_levels_array >= TIGHT_LOWER) & (glucose_levels_array <= TIGHT_UPPER)
    below_range = glucose_levels_array < LOWER_TARGET
    above_range = glucose_levels_array > UPPER_TARGET

    # Calculate percentages
    if len(glucose_levels_array) > 0:
        percentage_in_target = (np.sum(in_target_range) / len(glucose_levels_array)) * 100
        percentage_in_tight = (np.sum(in_tight_range) / len(glucose_levels_array)) * 100
        percentage_below = (np.sum(below_range) / len(glucose_levels_array)) * 100
        percentage_above = (np.sum(above_range) / len(glucose_levels_array)) * 100
    else:
        percentage_in_target = percentage_in_tight = percentage_below = percentage_above = 0

    return percentage_in_target, percentage_in_tight, percentage_below, percentage_above

# Add percentage times to the DataFrame
GlucoseEvents_exploded_clean[['Percentage Time in Target Range',
                              'Percentage Time in Tight Range',
                              'Percentage Time Below Range',
                              'Percentage Time Above Range']] = GlucoseEvents_exploded_clean['GlucoseLevels'].apply(
    lambda levels: pd.Series(calculate_time_in_ranges(levels))
)

# Group by MealCategory and GroupNumber to calculate metrics
grouped_stats = (
    GlucoseEvents_exploded_clean
    .groupby(['MealCategory', 'GroupNumber'])
    .agg(
        MeanPercentageTimeInTarget=('Percentage Time in Target Range', 'mean'),
        StdPercentageTimeInTarget=('Percentage Time in Target Range', 'std'),
        MeanPercentageTimeInTight=('Percentage Time in Tight Range', 'mean'),
        StdPercentageTimeInTight=('Percentage Time in Tight Range', 'std'),
        MeanPercentageTimeBelow=('Percentage Time Below Range', 'mean'),
        StdPercentageTimeBelow=('Percentage Time Below Range', 'std'),
        MeanPercentageTimeAbove=('Percentage Time Above Range', 'mean'),
        StdPercentageTimeAbove=('Percentage Time Above Range', 'std'),
        Count=('GlucoseLevels', 'count')  # Number of events in the group
    )
    .reset_index()
)

# Display grouped metrics
print(grouped_stats)


  MealCategory  GroupNumber  MeanPercentageTimeInTarget  \
0    Breakfast           -1                   77.604167   
1    Breakfast            2                   79.353933   
2       Dinner           -1                   69.270833   
3       Dinner            1                   58.564815   
4       Dinner            2                   83.946078   
5        Lunch           -1                   65.625000   
6        Lunch            2                   73.125000   
7        Snack           -1                   54.583333   
8        Snack            1                   70.258621   
9        Snack            2                   46.527778   

   StdPercentageTimeInTarget  MeanPercentageTimeInTight  \
0                  13.753682                  48.958333   
1                  12.052601                  51.872659   
2                  22.250416                  46.744792   
3                  23.337747                  32.539683   
4                  15.108920                  74.387255

In [ ]:
# Constants for target glucose range
LOWER_TARGET = 3.9  # Lower limit of target range (mmol/L)
UPPER_TARGET = 10.0  # Upper limit of target range (mmol/L)

# Function to calculate the percentage time in target range
def calculate_time_in_target_range(glucose_levels):
    # Ensure the input is a pandas Series to handle non-numeric entries
    if isinstance(glucose_levels, np.ndarray):
        glucose_levels_series = pd.Series(glucose_levels)
    else:
        glucose_levels_series = pd.Series(glucose_levels)

    # Convert glucose levels to numeric values, forcing invalid strings to NaN
    glucose_levels_series = pd.to_numeric(glucose_levels_series, errors='coerce')

    # Drop any NaN values that result from invalid conversions
    glucose_levels_clean = glucose_levels_series.dropna()

    # Convert to numpy array for further calculations
    glucose_levels_array = glucose_levels_clean.to_numpy()

    # Create a mask for values within the target range
    in_target_range = (glucose_levels_array >= LOWER_TARGET) & (glucose_levels_array <= UPPER_TARGET)

    # Calculate percentage of time in target range, ensure length is not zero
    if len(glucose_levels_array) > 0:
        percentage_time_in_target = (np.sum(in_target_range) / len(glucose_levels_array)) * 100
    else:
        percentage_time_in_target = 0

    return percentage_time_in_target

# Initialize an empty list to store percentage time in target range
percentage_time_in_target_range = []

# Iterate over each row in the GlucoseEvents DataFrame to calculate percentage time in target range
for index, row in GlucoseEvents_exploded_clean.iterrows():
    glucose_levels = row['GlucoseLevels']

    # Calculate the percentage time in target range for the glucose levels
    percentage = calculate_time_in_target_range(glucose_levels)

    # Store the result
    percentage_time_in_target_range.append(percentage)

# Add the percentage time in target range to the DataFrame
GlucoseEvents_exploded_clean['Percentage Time in Target Range'] = percentage_time_in_target_range

# Count the number of times each MealTag is used
meal_tag_counts = GlucoseEvents_exploded_clean['MealTag'].value_counts()

# Print or further analyze the counts
print("Number of times each MealTag is used:")
print(meal_tag_counts)

# Count how many MealTag counts are more than 1
meal_tag_counts_more_than_1 = (meal_tag_counts > 1).sum()
print(f"Number of Meal_tag_counts more than 1: {meal_tag_counts_more_than_1}")

# Count the total number of unique MealTags
total_unique_meal_tags = meal_tag_counts.count()
print(f"Total number of unique Meal_tags: {total_unique_meal_tags}")

# Calculate mean and standard deviation for time in target range
mean_time_in_target_range = GlucoseEvents_exploded_clean['Percentage Time in Target Range'].mean()
std_time_in_target_range = GlucoseEvents_exploded_clean['Percentage Time in Target Range'].std()

# Print the results
print(f"Mean Percentage Time in Target Range: {mean_time_in_target_range:.2f}%")
print(f"Standard Deviation of Percentage Time in Target Range: {std_time_in_target_range:.2f}%")


# Function to calculate mean, standard deviation, and count of events above a threshold
def calculate_time_in_target_stats(df, column_name="Percentage Time in Target Range", threshold=75):
    if column_name not in df.columns:
        raise KeyError(f"'{column_name}' not found in DataFrame.")

    mean_time_in_target = df[column_name].mean()
    std_time_in_target = df[column_name].std()
    count_above_threshold = (df[column_name] >= threshold).sum()  # Count events meeting the threshold
    return mean_time_in_target, std_time_in_target, count_above_threshold

# Calculate for GlucoseEvents DataFrame, using 'Percentage Time in Target Range' column
try:
    original_mean, original_std, original_count = calculate_time_in_target_stats(GlucoseEvents_exploded_clean)
    print(f"Glucose Events - Mean Time in Target: {original_mean:.2f}%, Standard Deviation: {original_std:.2f}%, Count above 75%: {original_count}")
except KeyError as e:
    print(e)

# Group data by MealCategory and GroupNumber to compute cluster-level statistics
cluster_time_in_range_stats = (
    GlucoseEvents_exploded_clean
    .groupby(['MealCategory', 'GroupNumber'])
    .agg(
        MeanTimeInRange=('Percentage Time in Target Range', 'mean'),
        StdTimeInRange=('Percentage Time in Target Range', 'std'),
        Count=('Percentage Time in Target Range', 'count')
    )
    .reset_index()
)

# Merge these stats back into cohesion_metrics_with_anova
cohesion_metrics_with_anova = pd.merge(
    cohesion_metrics_with_anova,
    cluster_time_in_range_stats,
    on=['MealCategory', 'GroupNumber'],
    how='left'
)

# Print final DataFrame for validation
print(cohesion_metrics_with_anova)


Number of times each MealTag is used:
MealTag
Porridge                 30
Granola + Skyr2          13
Granola +  Skyr           6
Tuna Sandwich             6
Granola + yoghurt         5
                         ..
Cheesecake                1
Oreo Milkshake            1
Cocktail + Cider          1
IceCream                  1
Peanutbutter Icecream     1
Name: count, Length: 187, dtype: int64
Number of Meal_tag_counts more than 1: 51
Total number of unique Meal_tags: 187
Mean Percentage Time in Target Range: 71.35%
Standard Deviation of Percentage Time in Target Range: 18.36%
Glucose Events - Mean Time in Target: 71.35%, Standard Deviation: 18.36%, Count above 75%: 170
    MeanPeak  StandardDeviation  CoefficientOfVariation  StandardError  \
0        NaN                NaN                     NaN            NaN   
1   9.388016           2.632589               28.042012       0.040278   
2   9.453231           4.359694               46.118560       0.629268   
3  11.400714           3.4682

#####################################

SIMULATIONS

#####################################

In [ ]:

# Load Dataset
df = GlucoseEvents_exploded_clean  # Replace with your actual dataset
df["NetGlucose"] = df["NetGlucose"].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
df.dropna(subset=["NetGlucose"], inplace=True)

# Constants and Parameters
ISF = insulin_sensitivity_factor  # Insulin Sensitivity Factor
GLUCOSE_TARGET_MIN = 3.9
GLUCOSE_TARGET_MAX = 10.0
VG = 12.0  # Volume of distribution of glucose
tau_m = 40  # Glucose metabolism time constant
GEZI = 0.01  # Glucose effectiveness factor
tau1 = 50  # Insulin sensitivity time constant 1
tau2 = 60  # Insulin sensitivity time constant 2

# Convert mg/dL to mmol/L
def mgdl_to_mmolL(glucose_mgdl):
    return glucose_mgdl / 18

# Improved Insulin Pharmacokinetics Function
def insulin_pharmacokinetics(t, insulin_doses, bolus_offsets):
    insulin_effect = 0
    for dose, bolus_offset in zip(insulin_doses, bolus_offsets):
        t_shifted = t - bolus_offset
        if t_shifted < 0:
            continue
        T_peak = 60  # Peak insulin action (min)
        T_half = 120  # Half-life (min)
        alpha = 1.5  # Peak sharpness
        beta = 0.05  # Secondary decay factor (new)

        # Dual-phase insulin effect
        primary_phase = dose * ((t_shifted / T_peak) ** alpha) * np.exp(-t_shifted / T_half)
        secondary_phase = dose * np.exp(-beta * t_shifted)
        insulin_effect += primary_phase + secondary_phase * 0.3  # 30% weight to secondary phase
    return insulin_effect

# EGP function with improved suppression factor
def endogenous_glucose_production(IEFF, baseline_egp=2.0, suppression_factor=0.7):
    return baseline_egp * np.exp(-suppression_factor * IEFF)

# Glucose model with better insulin integration
def glucose_model(y, t, insulin_doses, bolus_offsets, learned_glucose, max_change_rate, ISF):
    ISC, IP, IEFF, G = y
    insulin_input = insulin_pharmacokinetics(t, insulin_doses, bolus_offsets)

    # Insulin compartments
    dISC_dt = (-ISC / tau1) + (insulin_input / tau1)
    dIP_dt = (-IP / tau2) + (ISC / tau2)
    dIEFF_dt = (-IEFF / tau_m) + (IP / tau_m)

    # Endogenous glucose production suppression by insulin
    dynamic_EGP = endogenous_glucose_production(IEFF)

    # Glucose appearance from meal
    glucose_idx = min(int(t), len(learned_glucose) - 1)
    learned_glucose_value = learned_glucose[glucose_idx]
    max_glucose_change = max_change_rate[glucose_idx]
    RA = (learned_glucose_value / VG) * ((t / tau_m**2) * np.exp(-t / tau_m))

    # Updated glucose dynamics including insulin sensitivity explicitly
    dG_dt = -(GEZI + (IEFF / ISF)) * G + dynamic_EGP + RA

    return [dISC_dt, dIP_dt, dIEFF_dt, dG_dt]


#######################

Simulation by Cluster Alone

######################

In [ ]:

# Convert glucose response from string/list to array
def convert_glucose_levels(entry):
    if isinstance(entry, str):
        try:
            return np.array(eval(entry), dtype=float)
        except:
            return None
    elif isinstance(entry, list) or isinstance(entry, np.ndarray):
        return np.array(entry, dtype=float)
    return None


# Ensure simulated glucose follows learned response rate
def constrain_glucose_change(simulated_glucose, learned_glucose, max_change_rate, relaxation_factor=1.2):
    min_length = min(len(simulated_glucose), len(learned_glucose), len(max_change_rate))
    constrained_glucose = [simulated_glucose[0]]

    for i in range(1, min_length):
        max_change = max_change_rate[i] * relaxation_factor
        learned_value = learned_glucose[i]
        lower_bound = learned_value - max_change
        upper_bound = learned_value + max_change

        # Factor in insulin effectiveness to allow glucose to drop further if insulin is high
        constrained_value = np.clip(simulated_glucose[i], lower_bound * (1 - relaxation_factor * 0.1), upper_bound)
        constrained_glucose.append(constrained_value)

    return np.array(constrained_glucose)

def learn_glucose_response(df):
    response_patterns = {}
    grouped = df.groupby(["MealCategory", "GroupNumber"])["NetGlucose"]

    for (meal_category, cluster_id), glucose_series in grouped:
        glucose_arrays = [arr for arr in glucose_series if isinstance(arr, np.ndarray)]

        if len(glucose_arrays) > 0:
            # Find minimum common length
            min_length = min(len(arr) for arr in glucose_arrays)

            # Trim all arrays to the same length
            trimmed_arrays = [arr[:min_length] for arr in glucose_arrays]

            avg_response = np.mean(trimmed_arrays, axis=0)
            max_change_rate = np.abs(np.diff(avg_response, prepend=avg_response[0]))

            response_patterns[(meal_category, cluster_id)] = (avg_response, max_change_rate)

    return response_patterns

# Round glucose values
def round_glucose_values(glucose_values):
    return np.round(glucose_values, 2)


In [ ]:

# Calculate time in range
def calculate_time_in_range(glucose_levels):
    glucose_levels_array = np.array(glucose_levels)
    in_target_range = (glucose_levels_array >= GLUCOSE_TARGET_MIN) & (glucose_levels_array <= GLUCOSE_TARGET_MAX)
    return (np.sum(in_target_range) / len(glucose_levels_array)) * 100 if len(glucose_levels_array) > 0 else 0



#######################

Simulation by MealTag + Cluster

######################

In [ ]:


# # Optimization function
# def optimize_for_meal_clusters(df):
#     results = []
#     response_patterns = learn_glucose_response(df)

#     insulin_dose_options = list(range(1, 16))
#     bolus_offset_options = list(range(-45, 46, 15))

#     single_dose_combinations = [(dose,) for dose in insulin_dose_options]
#     two_dose_combinations = list(combinations(insulin_dose_options, 2))

#     single_offset_combinations = [(offset,) for offset in bolus_offset_options]
#     two_offset_combinations = list(combinations(bolus_offset_options, 2))

#     insulin_combinations = [(doses, offsets) for doses, offsets in product(single_dose_combinations, single_offset_combinations)]
#     insulin_combinations += [(doses, offsets) for doses, offsets in product(two_dose_combinations, two_offset_combinations)]

#     for (meal_category, cluster_id), (learned_glucose, max_change_rate) in response_patterns.items():
#         learned_time_in_range = calculate_time_in_range(learned_glucose)  # NEW: Time in Range for learned response

#         for insulin_doses, bolus_offsets in insulin_combinations:
#             t = np.linspace(0, 240, 240)
#             G0 = learned_glucose[0] if len(learned_glucose) > 0 else mgdl_to_mmolL(100)

#             initial_insulin_effect = insulin_pharmacokinetics(0, [insulin_doses[0]], [bolus_offsets[0]])
#             y0 = [initial_insulin_effect, initial_insulin_effect, 0, G0]

#             result = odeint(glucose_model, y0, t, args=(insulin_doses, bolus_offsets, learned_glucose, max_change_rate, ISF))
#             glucose_levels = result[:, 3]

#             glucose_levels_rounded = round_glucose_values(constrain_glucose_change(glucose_levels, learned_glucose, max_change_rate))

#             time_in_range = calculate_time_in_range(glucose_levels_rounded)

#             results.append({
#                 'Meal Category': meal_category,
#                 'Cluster': cluster_id,
#                 'Insulin Doses': list(insulin_doses),
#                 'Bolus Offsets': list(bolus_offsets),
#                 'Time in Range (%)': round(time_in_range, 2),
#                 'Learned Time in Range (%)': round(learned_time_in_range, 2),  # NEW: Added
#                 'Simulated Glucose Response': glucose_levels_rounded.tolist(),
#                 'Learned Glucose Response': learned_glucose.tolist()
#             })

#     return pd.DataFrame(results)

# # Run optimization and print results
# results_df = optimize_for_meal_clusters(df)
# print(results_df)



In [ ]:

# # Group the results by Meal Category and Cluster, and find the row with the highest Time in Range for each group
# best_results_df = results_df.loc[results_df.groupby(['Meal Category', 'Cluster'])['Time in Range (%)'].idxmax()]

# # Display the best results for each Meal Category / Cluster combination
# print(best_results_df)


In [ ]:
# # Ensure participant_id is defined
# participant_id = participant_file  # Replace with actual participant ID

# # Create a folder for saving plots
# output_folder = "plots"
# os.makedirs(output_folder, exist_ok=True)

# # Function to visualize the best glucose responses for each meal category
# def visualize_simulation_results(results_df):
#     if best_results_df.empty:
#         print("No valid simulation results to visualize.")
#         return

#     meal_categories = best_results_df["Meal Category"].unique()

#     for meal_category in meal_categories:
#         plt.figure(figsize=(10, 6))

#         meal_subset = best_results_df[results_df["Meal Category"] == meal_category]
#         time_points = np.linspace(0, 240, 240)

#         for _, row in meal_subset.iterrows():
#             cluster_id = row["Cluster"]
#             simulated_glucose = np.array(row["Simulated Glucose Response"])
#             learned_glucose = np.array(row["Learned Glucose Response"])

#             if len(learned_glucose) < 2:
#                 print(f"Skipping Cluster {cluster_id}: Not enough data points.")
#                 continue

#             # Interpolation to match 240 time points
#             learned_time_points = np.linspace(0, 240, len(learned_glucose))
#             interp_func = interp1d(learned_time_points, learned_glucose, kind='linear', fill_value="extrapolate")
#             learned_glucose_interp = interp_func(time_points)

#             if len(simulated_glucose) != 240:
#                 simulated_time_points = np.linspace(0, 240, len(simulated_glucose))
#                 sim_interp_func = interp1d(simulated_time_points, simulated_glucose, kind='linear', fill_value="extrapolate")
#                 simulated_glucose_interp = sim_interp_func(time_points)
#             else:
#                 simulated_glucose_interp = simulated_glucose

#             plt.plot(time_points, simulated_glucose_interp, label=f"Simulated - Cluster {cluster_id}", linewidth=2)
#             plt.plot(time_points, learned_glucose_interp, '--', label=f"Learned - Cluster {cluster_id}", linewidth=2)

#         plt.axhspan(3.9, 10, color='green', alpha=0.2, label="Time in Range (3.9-10 mmol/L)")
#         plt.title(f"Simulated vs Learned Glucose Responses for {meal_category}")
#         plt.xlabel("Time (minutes)")
#         plt.ylabel("Glucose Level (mmol/L)")
#         plt.legend()
#         plt.grid(True)
#         plt.show()

#         # Save the plot
#         plot_filename = f"{output_folder}/{participant_id}_{meal_category}_glucose_response.png"
#         plt.savefig(plot_filename, dpi=300)
#         plt.close()  # Close the plot to free memory

#         print(f"Plot saved: {plot_filename}")

# # Call function to visualize results
# visualize_simulation_results(best_results_df)



#######################

Simulation by MealTag Alone

######################

In [ ]:

def tag_insulin_pharmacokinetics(t, insulin_doses, bolus_offsets):
    insulin_effect = np.zeros_like(t)

    for dose, bolus_offset in zip(insulin_doses, bolus_offsets):
        t_shifted = t - bolus_offset  # Shift the time by bolus offset

        # Ensure only non-negative time values are considered
        valid_indices = t_shifted >= 0
        t_valid = t_shifted[valid_indices]

        if t_valid.size > 0:
            T_peak = 60  # Peak insulin action (minutes)
            action_profile = (t_valid / T_peak) * np.exp(1 - (t_valid / T_peak))
            insulin_effect[valid_indices] += dose * action_profile

    return insulin_effect

# # Learn glucose response for meal tags

def mealtag_learn_glucose_response(df):
    response_patterns = {}
    grouped = df.groupby("MealTag")["NetGlucose"]

    for meal_tag, glucose_series in grouped:
        glucose_arrays = []

        # Ensure each entry in glucose_series is iterable
        for arr in glucose_series:
            if isinstance(arr, (list, np.ndarray)):  # Check if it's iterable
                glucose_arrays.append(np.array(arr))  # Convert to numpy array for consistency
            else:
                glucose_arrays.append([])  # Replace non-iterable entries with empty lists

        if len(glucose_arrays) > 0:
            # Handle case where glucose_arrays is not empty
            max_length = max(map(len, glucose_arrays))  # Find the longest array length
            padded_arrays = [np.pad(arr, (0, max_length - len(arr)), mode='constant', constant_values=np.nan)
                             for arr in glucose_arrays]

            avg_response = np.nanmean(padded_arrays, axis=0)
            max_change_rate = np.abs(np.diff(avg_response, prepend=avg_response[0]))

            response_patterns[meal_tag] = (avg_response, max_change_rate)

    return response_patterns


# # def mealtag_learn_glucose_response(df):
# #     response_patterns = {}
# #     grouped = df.groupby("MealTag")["NetGlucose"]

# #     for meal_tag, glucose_series in grouped:
# #         glucose_arrays = [np.array(arr) for arr in glucose_series if arr is not None]

# #         if len(glucose_arrays) > 0:
# #             max_length = max(map(len, glucose_arrays))
# #             padded_arrays = [np.pad(arr, (0, max_length - len(arr)), mode='constant', constant_values=np.nan)
# #                              for arr in glucose_arrays]

# #             avg_response = np.nanmean(padded_arrays, axis=0)
# #             max_change_rate = np.abs(np.diff(avg_response, prepend=avg_response[0]))

# #             response_patterns[meal_tag] = (avg_response, max_change_rate)

# #     return response_patterns


In [ ]:

# # Ensure this function returns all meal tags in df, even if some are missing in response_patterns
# def optimize_for_meal_tags(df):
#     results = []
#     all_meal_tags = df['MealTag'].unique()
#     response_patterns = mealtag_learn_glucose_response(df)

#     insulin_dose_options = [2, 4, 6, 8, 10, 12, 14]
#     bolus_offset_options = list(range(-30, 31, 15))

#     two_dose_combinations = list(combinations(insulin_dose_options, 2))
#     two_offset_combinations = list(combinations(bolus_offset_options, 2))

#     insulin_combinations = list(product(two_dose_combinations, two_offset_combinations))

#     t = np.linspace(0, 240, 240)

#     for meal_tag in all_meal_tags:
#         if meal_tag in response_patterns:
#             learned_glucose, max_change_rate = response_patterns[meal_tag]
#         else:
#             learned_glucose = np.array([mgdl_to_mmolL(100)])
#             max_change_rate = np.full_like(t, 1)

#         learned_time_in_range = calculate_time_in_range(learned_glucose)  # NEW: Time in Range for learned response
#         G0 = learned_glucose[0] if len(learned_glucose) > 0 else mgdl_to_mmolL(100)

#         for (insulin_doses, bolus_offsets) in insulin_combinations:
#             insulin_effect = tag_insulin_pharmacokinetics(t, insulin_doses, bolus_offsets)

#             initial_insulin_effect = insulin_effect[0]
#             y0 = [initial_insulin_effect, initial_insulin_effect, 0, G0]

#             result = odeint(glucose_model, y0, t, args=(insulin_doses, bolus_offsets, learned_glucose, max_change_rate, ISF))

#             glucose_levels = result[:, 3]

#             glucose_levels_rounded = round_glucose_values(constrain_glucose_change(glucose_levels, learned_glucose, max_change_rate))

#             time_in_range = np.mean((glucose_levels_rounded >= GLUCOSE_TARGET_MIN) &
#                                     (glucose_levels_rounded <= GLUCOSE_TARGET_MAX)) * 100

#             total_insulin_dose = sum(insulin_doses)

#             results.append({
#                 'MealTag': meal_tag,
#                 'Insulin Doses': list(insulin_doses),
#                 'Bolus Offsets': list(bolus_offsets),
#                 'Total Insulin Dose': total_insulin_dose,
#                 'Time in Range (%)': round(time_in_range, 2),
#                 'Learned Time in Range (%)': round(learned_time_in_range, 2),  # NEW: Added
#                 'Simulated Glucose Response': glucose_levels_rounded.tolist(),
#                 'Learned Glucose Response': learned_glucose.tolist()
#             })

#     return pd.DataFrame(results)

# # Run optimized simulation
# mealtag_results_df = optimize_for_meal_tags(df)
# print(mealtag_results_df)



In [ ]:

# def get_best_simulations(results_df):
#     """
#     Filters the best simulations for each MealTag based on:
#     1. The highest Time in Range (TIR %)
#     2. The lowest total insulin dose (if multiple entries have the best TIR)
#     """
#     best_results = []

#     # Convert doses from string to tuple if necessary
#     if "Insulin Doses" in results_df.columns:
#         results_df["Insulin Doses"] = results_df["Insulin Doses"].apply(lambda x: tuple(x) if isinstance(x, list) else x)

#     for meal_tag, group in results_df.groupby("MealTag"):
#         # Get the maximum TIR%
#         max_tir = group["Time in Range (%)"].max()

#         # Filter rows that have this max TIR%
#         best_tir_group = group[group["Time in Range (%)"] == max_tir]

#         # Sort by the lowest Total Insulin Dose and keep all with the same min dose
#         min_dose = best_tir_group["Total Insulin Dose"].min()
#         best_simulations = best_tir_group[best_tir_group["Total Insulin Dose"] == min_dose]

#         best_results.append(best_simulations)

#     return pd.concat(best_results, ignore_index=True)

# # Run optimized simulation and filter best results
# mealtag_results_df = optimize_for_meal_tags(df)
# best_meal_tag_simulations_df = get_best_simulations(mealtag_results_df)

# # Print best simulations per MealTag
# print(best_meal_tag_simulations_df)



#########################

Meal Cat and Meal Tag Simulation

#########################

In [ ]:
from functools import partial
from itertools import product
from multiprocessing import Pool, cpu_count
from scipy.integrate import odeint


def learn_combined_glucose_response(df):
    response_patterns = {}
    grouped = df.groupby(["MealTag", "MealCategory", "GroupNumber"])["NetGlucose"]

    for (tag, category, cluster_id), glucose_series in grouped:
        glucose_arrays = [arr for arr in glucose_series if isinstance(arr, np.ndarray)]
        if glucose_arrays:
            min_len = min(len(arr) for arr in glucose_arrays)
            trimmed_arrays = [arr[:min_len] for arr in glucose_arrays]
            avg_response = np.mean(trimmed_arrays, axis=0)
            max_change_rate = np.abs(np.diff(avg_response, prepend=avg_response[0]))
            response_patterns[(tag, category, cluster_id)] = (avg_response, max_change_rate)

    return response_patterns

def simulate_single_combo(args, learned_glucose, max_change_rate, ISF):
    insulin_doses, bolus_offsets = args
    t = np.linspace(0, 240, 240)
    G0 = learned_glucose[0] if len(learned_glucose) > 0 else mgdl_to_mmolL(100)

    initial_insulin_effect = insulin_pharmacokinetics(0, insulin_doses, bolus_offsets)
    y0 = [initial_insulin_effect, initial_insulin_effect, 0, G0]

    result = odeint(glucose_model, y0, t, args=(insulin_doses, bolus_offsets, learned_glucose, max_change_rate, ISF))
    glucose_levels = result[:, 3]
    glucose_levels_rounded = round_glucose_values(constrain_glucose_change(glucose_levels, learned_glucose, max_change_rate))
    time_in_range = calculate_time_in_range(glucose_levels_rounded)

    return {
        'Insulin Doses': insulin_doses,
        'Bolus Offsets': bolus_offsets,
        'Simulated Glucose Response': glucose_levels_rounded.tolist(),
        'Time in Range (%)': round(time_in_range, 2)
    }

def simulate_mealtag_across_categories(df, ISF):
    combined_patterns = learn_combined_glucose_response(df)
    all_results = []

    for (tag, category, cluster_id), (learned_glucose, max_change_rate) in combined_patterns.items():
        insulin_dose_options = range(2, 13, 2)
        bolus_offset_options = range(-30, 31, 15)

        dose_combinations = list(product(insulin_dose_options, repeat=2))
        offset_combinations = [(a, b) for a in bolus_offset_options for b in bolus_offset_options if abs(a - b) >= 15]
        insulin_combinations = list(product(dose_combinations, offset_combinations))

        with Pool(processes=cpu_count()) as pool:
            simulate_partial = partial(simulate_single_combo,
                                       learned_glucose=learned_glucose,
                                       max_change_rate=max_change_rate,
                                       ISF=ISF)
            simulation_outputs = pool.map(simulate_partial, insulin_combinations)

        for output in simulation_outputs:
            all_results.append({
                'MealTag': tag,
                'Meal Category': category,
                'Cluster': cluster_id,
                **output,
                'Learned Glucose Response': learned_glucose.tolist()
            })

    return pd.DataFrame(all_results)

def filter_best_simulations_with_improvement(simulation_results):
    simulation_results["Total Insulin Dose"] = simulation_results["Insulin Doses"].apply(sum)
    simulation_results["Learned Time in Range (%)"] = simulation_results["Learned Glucose Response"].apply(
        lambda glucose: round(calculate_time_in_range(round_glucose_values(glucose)), 2)
    )

    simulation_results_sorted = simulation_results.sort_values(
        by=["MealTag", "Meal Category", "Cluster", "Time in Range (%)", "Total Insulin Dose"],
        ascending=[True, True, True, False, True]
    )

    best_simulations = simulation_results_sorted.groupby(["MealTag", "Meal Category", "Cluster"], as_index=False).first()
    best_simulations["Improvement in Time in Range (%)"] = (
        best_simulations["Time in Range (%)"] - best_simulations["Learned Time in Range (%)"]
    )

    return best_simulations

# ---- EXECUTION ----

simulation_results = simulate_mealtag_across_categories(df, ISF)

if not simulation_results.empty:
    print(simulation_results[[
        "MealTag", "Meal Category", "Cluster", "Insulin Doses", "Bolus Offsets",
        "Time in Range (%)", "Learned Glucose Response", "Simulated Glucose Response"
    ]].head(10))

    best_simulations = filter_best_simulations_with_improvement(simulation_results)
    print(best_simulations[[
        "MealTag", "Meal Category", "Cluster",
        "Time in Range (%)", "Learned Time in Range (%)", "Improvement in Time in Range (%)",
        "Insulin Doses", "Bolus Offsets", "Total Insulin Dose"
    ]].head())
else:
    print("No simulation results met the 70% time-in-range threshold.")


    MealTag Meal Category  Cluster Insulin Doses Bolus Offsets  \
0  All Bran     Breakfast        2        (2, 2)    (-30, -15)   
1  All Bran     Breakfast        2        (2, 2)      (-30, 0)   
2  All Bran     Breakfast        2        (2, 2)     (-30, 15)   
3  All Bran     Breakfast        2        (2, 2)     (-30, 30)   
4  All Bran     Breakfast        2        (2, 2)    (-15, -30)   
5  All Bran     Breakfast        2        (2, 2)      (-15, 0)   
6  All Bran     Breakfast        2        (2, 2)     (-15, 15)   
7  All Bran     Breakfast        2        (2, 2)     (-15, 30)   
8  All Bran     Breakfast        2        (2, 2)      (0, -30)   
9  All Bran     Breakfast        2        (2, 2)      (0, -15)   

   Time in Range (%)                           Learned Glucose Response  \
0              60.42  [11.3, 10.92624028727086, 9.894972780486436, 8...   
1              62.50  [11.3, 10.92624028727086, 9.894972780486436, 8...   
2              39.58  [11.3, 10.92624028727086, 

In [ ]:

simulation_results = simulate_mealtag_across_categories(df, ISF)

if not simulation_results.empty:
    print(simulation_results[[
        "MealTag", "Meal Category", "Cluster", "Insulin Doses", "Bolus Offsets",
        "Time in Range (%)", "Learned Glucose Response", "Simulated Glucose Response"
    ]].head(10))

    best_simulations = filter_best_simulations_with_improvement(simulation_results)
    print(best_simulations[[
        "MealTag", "Meal Category", "Cluster",
        "Time in Range (%)", "Learned Time in Range (%)", "Improvement in Time in Range (%)",
        "Insulin Doses", "Bolus Offsets", "Total Insulin Dose"
    ]].head())
else:
    print("No simulation results met the 70% time-in-range threshold.")


    MealTag Meal Category  Cluster Insulin Doses Bolus Offsets  \
0  All Bran     Breakfast        2        (2, 2)    (-30, -15)   
1  All Bran     Breakfast        2        (2, 2)      (-30, 0)   
2  All Bran     Breakfast        2        (2, 2)     (-30, 15)   
3  All Bran     Breakfast        2        (2, 2)     (-30, 30)   
4  All Bran     Breakfast        2        (2, 2)    (-15, -30)   
5  All Bran     Breakfast        2        (2, 2)      (-15, 0)   
6  All Bran     Breakfast        2        (2, 2)     (-15, 15)   
7  All Bran     Breakfast        2        (2, 2)     (-15, 30)   
8  All Bran     Breakfast        2        (2, 2)      (0, -30)   
9  All Bran     Breakfast        2        (2, 2)      (0, -15)   

   Time in Range (%)                           Learned Glucose Response  \
0              60.42  [11.3, 10.92624028727086, 9.894972780486436, 8...   
1              62.50  [11.3, 10.92624028727086, 9.894972780486436, 8...   
2              39.58  [11.3, 10.92624028727086, 

###################

Meal Category and Cluster ~ Pump Simulation

##################

In [ ]:
# ### Insulin Pump Simulation ~ Set to multiple doses at different time points through the simulation period ###

# # Basal Insulin Rate (U/min) - **New Addition**
# BASAL_INSULIN_RATE = 0.02  # Example: 0.02 U/min


# # Modified Insulin Pharmacokinetics Function (Multiple doses + Basal)
# def insulin_pharmacokinetics(t, insulin_doses, bolus_offsets, basal_rate):
#     insulin_effect = basal_rate  # Start with basal infusion effect

#     for dose, bolus_offset in zip(insulin_doses, bolus_offsets):
#         t_shifted = t - bolus_offset
#         if t_shifted < 0:
#             continue

#         peak_time = 60
#         duration = 180
#         insulin_effect += dose * np.exp(-((t_shifted - peak_time) ** 2) / (2 * (duration / 2) ** 2))

#     return insulin_effect


In [ ]:

# # Updated Glucose Model (Supports Multiple Insulin Doses)
# def pump_glucose_model(y, t, learned_glucose, max_change_rate, ISF, insulin_doses, bolus_offsets, basal_rate):
#     ISC, IP, IEFF, G = y
#     insulin_input = insulin_pharmacokinetics(t, insulin_doses, bolus_offsets, basal_rate)

#     # Insulin kinetics
#     dISC_dt = (-ISC / tau1) + (insulin_input / tau1)
#     dIP_dt = (-IP / tau2) + (ISC / tau2)
#     dIEFF_dt = (-IEFF / tau_m) + (IP / tau_m)

#     # Endogenous glucose production suppression by insulin
#     dynamic_EGP = endogenous_glucose_production(IEFF)

#     # Meal glucose absorption
#     glucose_idx = min(int(t), len(learned_glucose) - 1)
#     learned_glucose_value = learned_glucose[glucose_idx]
#     max_glucose_change = max_change_rate[glucose_idx]
#     RA = (learned_glucose_value / VG) * ((t / tau_m**2) * np.exp(-t / tau_m))

#     # Updated glucose dynamics including insulin sensitivity explicitly
#     dG_dt = -(GEZI + (IEFF / ISF)) * G + dynamic_EGP + RA

#     return [dISC_dt, dIP_dt, dIEFF_dt, dG_dt]

# # Modify the function to include 'Total Insulin Dose' in results
# def pump_optimize_for_meal_clusters(df, basal_rate=BASAL_INSULIN_RATE):
#     results = []
#     response_patterns = learn_glucose_response(df)

#     insulin_dose_schedules = [
#         ([1, 0.5, 1.5], [0, 60, 120]),
#         ([1, 1, 1], [0, 90, 180]),
#         ([2, 1], [0, 120])
#     ]

#     for (meal_category, cluster_id), (learned_glucose, max_change_rate) in response_patterns.items():
#         t = np.linspace(0, 240, 240)
#         G0 = learned_glucose[0] if len(learned_glucose) > 0 else mgdl_to_mmolL(100)

#         learned_time_in_range = calculate_time_in_range(learned_glucose)  # NEW: Time in Range for learned response

#         for insulin_doses, bolus_offsets in insulin_dose_schedules:
#             y0 = [0, 0, 0, G0]

#             result = odeint(pump_glucose_model, y0, t, args=(learned_glucose, max_change_rate, ISF, insulin_doses, bolus_offsets, basal_rate))
#             glucose_levels = result[:, 3]

#             glucose_levels_rounded = round_glucose_values(constrain_glucose_change(glucose_levels, learned_glucose, max_change_rate))

#             time_in_range = calculate_time_in_range(glucose_levels_rounded)
#             total_insulin_dose = sum(insulin_doses)

#             results.append({
#                 'Meal Category': meal_category,
#                 'Cluster': cluster_id,
#                 'Basal Rate (U/min)': basal_rate,
#                 'Insulin Doses': insulin_doses,
#                 'Bolus Offsets': bolus_offsets,
#                 'Total Insulin Dose': total_insulin_dose,
#                 'Time in Range (%)': round(time_in_range, 2),
#                 'Learned Time in Range (%)': round(learned_time_in_range, 2),  # NEW: Added
#                 'Simulated Glucose Response': glucose_levels_rounded.tolist(),
#                 'Learned Glucose Response': learned_glucose.tolist()
#             })

#     return pd.DataFrame(results)

# # Run the optimization with multiple doses + continuous basal insulin
# pump_results_df = pump_optimize_for_meal_clusters(df)

# print(pump_results_df)


In [ ]:
# # **Find the best strategy**
# max_time_in_range = pump_results_df["Time in Range (%)"].max()
# best_pump_results_df = pump_results_df[pump_results_df["Time in Range (%)"] == max_time_in_range]

# # Among those, choose the one with the smallest total insulin dose
# best_pump_results_df = best_pump_results_df[
#     best_pump_results_df["Total Insulin Dose"] == best_pump_results_df["Total Insulin Dose"].min()]


In [ ]:

# # **Visualization Function**
# def pump_visualize_simulation_results(results_df):
#     if results_df.empty:
#         print("No valid simulation results to visualize.")
#         return

#     meal_categories = results_df["Meal Category"].unique()
#     time_points = np.linspace(0, 240, 240)  # Standard time points

#     for meal_category in meal_categories:
#         plt.figure(figsize=(10, 6))
#         meal_subset = results_df[results_df["Meal Category"] == meal_category]

#         for _, row in meal_subset.iterrows():
#             cluster_id = row["Cluster"]
#             simulated_glucose = np.array(row["Simulated Glucose Response"])
#             learned_glucose = np.array(row["Learned Glucose Response"])

#             # Ensure learned_glucose is interpolated to 240 points
#             learned_time_points = np.linspace(0, 240, len(learned_glucose))
#             learned_interp_func = interp1d(learned_time_points, learned_glucose, kind='linear', fill_value="extrapolate")
#             learned_glucose_interp = learned_interp_func(time_points)

#             # Ensure simulated_glucose is interpolated to 240 points if needed
#             if len(simulated_glucose) != 240:
#                 simulated_time_points = np.linspace(0, 240, len(simulated_glucose))
#                 sim_interp_func = interp1d(simulated_time_points, simulated_glucose, kind='linear', fill_value="extrapolate")
#                 simulated_glucose_interp = sim_interp_func(time_points)
#             else:
#                 simulated_glucose_interp = simulated_glucose

#             plt.plot(time_points, simulated_glucose_interp, label=f"Simulated - Cluster {cluster_id}", linewidth=2)
#             plt.plot(time_points, learned_glucose_interp, '--', label=f"Learned - Cluster {cluster_id}", linewidth=2)

#         plt.axhspan(3.9, 10, color='green', alpha=0.2, label="Time in Range (3.9-10 mmol/L)")
#         plt.title(f"Simulated vs Learned Glucose Responses for {meal_category}")
#         plt.xlabel("Time (minutes)")
#         plt.ylabel("Glucose Level (mmol/L)")
#         plt.legend()
#         plt.grid(True)
#         plt.show()

# # **Run Visualization for the Best Strategies**
# pump_visualize_simulation_results(best_pump_results_df)


In [ ]:
pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Define the output Excel file name
output_file_name = f"{participant_file}_mealtag_cat_sim.xlsx"

# Save all the dataframes into a single Excel file, each on a separate sheet
with pd.ExcelWriter(output_file_name, engine='xlsxwriter') as writer:
    GlucoseEvents_exploded_clean.to_excel(writer, sheet_name='GlucoseEvents', index=True)
    grouped_stats.to_excel(writer, sheet_name='GroupStats', index=True)
    # summary_df.to_excel(writer, sheet_name='Baseline stats')
    cohesion_metrics_with_anova.to_excel(writer, sheet_name='CohesionMetricsWithANOVA', index=True)
    simulation_results.to_excel(writer, sheet_name="tags+MealCatSim", index=True)
    best_simulations.to_excel(writer, sheet_name="bestMealTagCatResults", index=True)
    # mealtag_results_df.to_excel(writer, sheet_name='MealTagSim', index=True)
    # best_meal_tag_simulations_df.to_excel(writer, sheet_name='best_MealTagSim', index=True)
    # results_df.to_excel(writer, sheet_name='MealCat_Cluster_Sim', index=True)
    # best_results_df.to_excel(writer, sheet_name='Best_MealCat_ClusteSim', index=True)
    # pump_results_df.to_excel(writer, sheet_name='Cat_Cluster_Pump', index=True)
    # best_pump_results_df.to_excel(writer, sheet_name='Best_Cat_Cluster_Pump', index=True)

# Confirmation message
print(f"Data saved successfully to {output_file_name}")



Data saved successfully to UoM2404.csv_mealtag_cat_sim.xlsx
